In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
# wget https://kaggle2.blob.core.windows.net/competitions-data/kaggle/3004/test.csv
# wget wget https://kaggle2.blob.core.windows.net/competitions-data/kaggle/3004/train.csv

In [176]:
csv = pd.read_csv("train.csv")
csv_test = pd.read_csv("test.csv")


In [178]:
csv_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
csv.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
def to_one_hot(y):
    return (np.arange(np.unique(y).__len__()) == y[:,None]).astype(float)

In [179]:
X = csv.iloc[:,1:].values.astype(np.float32)
X = np.multiply(X, 1.0 / 255.0)
y = csv.iloc[:,0].values
y_oh = to_one_hot(y)

X_test = csv_test.iloc[:,:].values.astype(np.float32)

In [180]:
print(X.shape, y.shape, y_oh.shape, X_test.shape)

(42000, 784) (42000,) (42000, 10) (28000, 784)


In [132]:
image_size = X.shape[1]
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
print("image_height/image_width = {}/{}".format(image_width, image_height))

image_height/image_width = 28/28


In [167]:
#VALIDATION_SIZE = 2000


# TF graph

In [109]:
BATCH_SIZE = 16

In [110]:
def wb(wshape=[None], bshape=[None], device='/cpu:0'):
    with tf.device(device):
        w = tf.get_variable("w", wshape, initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable('b', bshape, initializer=tf.constant_initializer(0.0))
    return w, b

In [111]:
def conv2d(x, W):
    # W => (patch, patch, depth_from, depth_to)
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [194]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
train_size = len(X)

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, 28, 28, 1))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, 10))
  tf_test_dataset = tf.constant(X_test.reshape((-1, 28, 28, 1)))
  
  # Variables.
  with tf.variable_scope("convNetConvLayer1"):
        layer1_weights, layer1_biases = wb([3, 3, 1, 16], [16])
  with tf.variable_scope("convNetConvLayer2"):
        layer2_weights, layer2_biases = wb([3, 3, 16, 128], [128])
  with tf.variable_scope("convNetFC1"):
        layer3_weights, layer3_biases = wb([image_width // 4 * image_width // 4 * 128, 1024], [1024])
  with tf.variable_scope("convNetFC2"):
        layer4_weights, layer4_biases = wb([1024, 10], [10])
  
  # Model with max_pool.
  def model(data, train=True):
    
    conv1 = tf.nn.relu(conv2d(data, layer1_weights) + layer1_biases)
    pool1 = max_pool_2x2(conv1)
    print("pool1", pool1.get_shape())
    
    conv2 = tf.nn.relu(conv2d(pool1, layer2_weights) + layer2_biases)
    pool2 = max_pool_2x2(conv2)
    print("hidden2", pool2.get_shape())
    
    shape = pool2.get_shape().as_list()
    reshape = tf.reshape(pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    
    fc1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    if train:
        tf.nn.dropout(fc1, 0.5)
    fc2 = tf.matmul(fc1, layer4_weights) + layer4_biases
    return fc2
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  learn_rate  = tf.train.exponential_decay(.05, global_step*batch_size, train_size, 0.5, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  test_prediction = tf.nn.softmax(model(tf_test_dataset, train=False))
    

pool1 (16, 14, 14, 16)
hidden2 (16, 7, 7, 128)
pool1 (28000, 14, 14, 16)
hidden2 (28000, 7, 7, 128)


In [195]:

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [ ]:
steps = train_size//batch_size * 3
print(steps)
ans = []

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for i in range(steps):
        offset = (i * batch_size) % (y_train.shape[0] - batch_size)
        batch_data = X_train[offset:(offset + batch_size), :]
        batch_labels = y_train[offset:(offset + batch_size), :]
        feed_dict = {
            tf_train_dataset : batch_data.reshape((-1, 28, 28, 1)), 
            tf_train_labels : batch_labels
        }

        _, l, pred = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (i < 100 and i % 10 == 0) or i % 100 == 0:
            print( "step: {}, accuracy: {}, loss: {}".format(i, accuracy(pred,batch_labels ), l, ))
    ans = test_prediction.eval()

7875
step: 0, accuracy: 0.0, loss: 2.5334978103637695
step: 10, accuracy: 31.25, loss: 1.8312147855758667
step: 20, accuracy: 50.0, loss: 2.5443167686462402
step: 30, accuracy: 75.0, loss: 0.7150152921676636
step: 40, accuracy: 68.75, loss: 0.8332014679908752
step: 50, accuracy: 87.5, loss: 0.41016390919685364
step: 60, accuracy: 93.75, loss: 0.2587459087371826
step: 70, accuracy: 100.0, loss: 0.16393707692623138
step: 80, accuracy: 56.25, loss: 1.1925885677337646
step: 90, accuracy: 81.25, loss: 0.5644762516021729
step: 100, accuracy: 93.75, loss: 0.24004116654396057
step: 200, accuracy: 100.0, loss: 0.025936398655176163
step: 300, accuracy: 100.0, loss: 0.052772264927625656
step: 400, accuracy: 93.75, loss: 0.5213586091995239
step: 500, accuracy: 93.75, loss: 0.12650848925113678
step: 600, accuracy: 93.75, loss: 0.1898028552532196
step: 700, accuracy: 100.0, loss: 0.046877436339855194
step: 800, accuracy: 100.0, loss: 0.06260539591312408
step: 900, accuracy: 100.0, loss: 0.0434478111

In [184]:
ans = np.argmax(ans, 1)

In [185]:
ans.shape

(28000,)

In [188]:
import csv
with open('answer.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["ImageId","Label"])
    for i, a in enumerate(ans):
        writer.writerow([i+1,a])